[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/sec-company-data-python.ipynb)

# SEC Company Data with Python -- Free, No API Key

Use **edgartools** to look up company data from SEC EDGAR -- completely free, no API key or paid subscription required. Get company metadata, filing history, financial statements, and industry classification for any SEC-registered entity by ticker symbol or CIK number.

**What you'll learn:**
- Look up any company by ticker symbol or CIK number
- Get company metadata: industry, address, incorporation, filer category
- Browse a company's complete filing history
- Access financial data directly from the Company object
- Search for companies by name
- Compare company profiles across industries

## Install edgartools

In [ ]:
!pip install -U edgartools

## Setup

The SEC requires all automated tools to identify themselves. Replace the email below with your own -- any valid email works.

In [ ]:
import pandas as pd
from edgar import *

# The SEC requires you to identify yourself (any email works)
set_identity("your.name@example.com")

## Look Up a Company in One Line

Pass a ticker symbol to `Company()` and get a rich data object with everything the SEC knows about that entity:

In [ ]:
company = Company("AAPL")
company

## Company Metadata

Every SEC-registered entity has metadata including industry classification, filer category, and contact information. Access these as Python properties:

In [ ]:
company = Company("AAPL")

print(f"Name:            {company.name}")
print(f"CIK:             {company.cik}")
print(f"Tickers:         {company.tickers}")
print(f"SIC Code:        {company.sic}")
print(f"Industry:        {company.industry}")
print(f"Fiscal Year End: {company.fiscal_year_end}")
print(f"Filer Category:  {company.filer_category}")
print(f"Business Type:   {company.business_category}")
print(f"Filer Type:      {company.filer_type}")
print(f"Phone:           {company.data.phone}")
print(f"Incorporated:    {company.data.state_of_incorporation}")
print(f"Address:         {company.business_address()}")

## Look Up by CIK Number

You can also look up a company by its CIK (Central Index Key) -- the unique identifier the SEC assigns to every filer:

In [ ]:
# Look up by integer CIK
apple = Company(320193)
print(f"{apple.name} ({apple.tickers})")

# Also works with zero-padded string CIK
tesla = Company("0001318605")
print(f"{tesla.name} ({tesla.tickers})")

## Browse Filing History

Every company has a complete filing history on EDGAR. Browse all filings or filter by form type:

In [ ]:
company = Company("AAPL")

all_filings = company.get_filings()
print(f"Total filings: {len(all_filings):,}\n")

# Count filings by form type
df = all_filings.to_pandas()
counts = df["form"].value_counts().head(10)

print(f"{'Form':<15s} {'Count':>6s}")
print(f"{'-'*15} {'-'*6}")
for form, count in counts.items():
    print(f"{form:<15s} {count:>6,}")

## Access Financial Data

Get key financial metrics directly from the Company object -- no need to parse individual filings:

In [ ]:
financials = Company("AAPL").get_financials()

print(f"Revenue:       ${financials.get_revenue():>18,.0f}")
print(f"Net Income:    ${financials.get_net_income():>18,.0f}")
print(f"Total Assets:  ${financials.get_total_assets():>18,.0f}")

financials.income_statement()

## Search for Companies by Name

Don't know the ticker? Use `find()` to search SEC EDGAR for companies by name:

In [ ]:
results = find("Tesla")
print(results)

print("\n--- Broader search ---\n")

results = find("bank")
print(results)

## Former Names

Companies change names over time. The SEC tracks every former name, which is useful for historical research:

In [ ]:
for ticker, label in [("AAPL", "Apple"), ("META", "Meta")]:
    company = Company(ticker)
    names = company.data.former_names
    print(f"{label} ({company.name}):")
    for entry in names:
        print(f"  {entry['name']:30s}  ({entry['from']} to {entry['to']})")
    print()

## Compare Company Profiles Across Industries

Pull metadata from companies across different sectors to compare their SEC profiles:

In [ ]:
tickers = ["AAPL", "MSFT", "TSLA", "JPM", "JNJ"]
rows = []

for ticker in tickers:
    c = Company(ticker)
    rows.append({
        "Ticker": ticker,
        "Name": c.name,
        "Industry": c.industry,
        "Filer Category": c.filer_category,
        "Total Filings": f"{len(c.get_filings()):,}",
    })

pd.DataFrame(rows).set_index("Ticker")

## Why EdgarTools?

EdgarTools is free and open-source. Compare accessing SEC company data:

**With edgartools (free, no API key):**
```python
company = Company("AAPL")
company.name                    # Company name
company.industry                # Industry classification
company.get_filings(form="10-K") # All 10-K filings
company.get_financials()         # Financial statements
find("Tesla")                    # Search by name
```

**Typical paid API approach ($50+/month, API key required):**
```python
from sec_api import QueryApi
api = QueryApi(api_key="YOUR_PAID_API_KEY")
query = {"query": {"query_string": {"query": 'ticker:AAPL'}}}
results = api.get_filings(query)  # Raw JSON, no company metadata
```

With edgartools, company data is a first-class Python object -- metadata, filings, and financials all accessible from a single `Company()` call.

## Quick Reference

```python
from edgar import *
set_identity("your.name@example.com")

# ── Look up a company ──
company = Company("AAPL")           # By ticker
company = Company(320193)            # By CIK (integer)
company = Company("0000320193")      # By CIK (string)

# ── Company metadata ──
company.name                         # Company name
company.cik                          # CIK number
company.tickers                      # Ticker symbols
company.sic                          # SIC code
company.industry                     # Industry description
company.fiscal_year_end              # Fiscal year end
company.filer_category               # SEC filer category
company.business_category            # Business type
company.data.phone                   # Phone number
company.data.former_names            # Historical names
company.business_address()           # Business address

# ── Filing history ──
company.get_filings()                # All filings
company.get_filings(form="10-K")     # Filter by form type
company.latest("10-K")              # Most recent 10-K

# ── Financial data ──
financials = company.get_financials()
financials.get_revenue()             # Revenue
financials.get_net_income()          # Net income
financials.income_statement()        # Full income statement

# ── Search ──
find("Tesla")                        # Search by company name
```

## What's Next

You've learned how to look up and explore SEC company data with Python. Here are related tutorials:

- [SEC EDGAR API in Python](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/sec-edgar-api-python.ipynb)
- [Search and Filter SEC Filings](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/search-sec-filings-python.ipynb)
- [Extract Financial Statements from SEC Filings](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/financial-statements-sec-python.ipynb)
- [Analyze 10-K Annual Reports](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/analyze-10k-annual-report-python.ipynb)

**Resources:**
- [EdgarTools Documentation](https://edgartools.readthedocs.io/)
- [GitHub Repository](https://github.com/dgunning/edgartools)
- [PyPI Package](https://pypi.org/project/edgartools/)

---

## Support EdgarTools

If you found this tutorial helpful, here are a few ways to support the project:

- **Star the repo** -- [github.com/dgunning/edgartools](https://github.com/dgunning/edgartools) -- it helps others discover edgartools
- **Visit edgartools.io** -- [edgartools.io](https://www.edgartools.io/) -- for more tutorials, articles, and updates
- **Report issues** -- found a bug or have a feature idea? [Open an issue](https://github.com/dgunning/edgartools/issues)
- **Share this notebook** -- know someone who works with SEC data? Send them the Colab link

*edgartools is free, open-source, and community-driven. No API key or paid subscription required.*